<a href="https://colab.research.google.com/github/DDiekmann/Applied-Verification-Lab-Neural-Networks/blob/main/Tutorials/Planet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial for Neural Network Verification using Planet



---

*As an example we try to verify the adversarial robustness of a classification Network trained on the MNIST dataset. The model is trained using [Caffe](https://caffe.berkeleyvision.org/) and the verification is done with [Planet](https://arxiv.org/abs/1705.01320).*

---

Planet is a powerful tool for the verification neural networks. However, the toolchain is quite outdated and not suitable for Google Colab. Neither Caffe nor Python 2.7 works in Colab. We managed to convert the scripts to Python 3, but the Caffe problem still remains. 

To elevate Planet from research software to becoming really useable, a converter from a standard format such as [ONNX](https://github.com/onnx/onnx) to Planet's .rlv input file is needed. This way, one could build networks with different frameworks and convert them to ONNX and then to Planet for verification


# Install Planet
[Github Repository](https://github.com/progirep/planet)

To verify neural networks with planet, we first clone the planet Repository from github to obtain planet.

In [ ]:
%%capture

# Clone the repo
!git clone https://github.com/progirep/planet.git

Next we intall the needed packages in order to build PLANET.

In [ ]:
%%capture

# install packages in order to build PLANET
!sudo apt-get install libglpk-dev
!sudo apt-get install qt5-qmake
!sudo apt-get install valgrind
!sudo apt-get install libltdl-dev
!sudo apt-get install protobuf-compiler

Now we can change into the src directory and trigger the build process with make.

In [ ]:
%%capture

# compile the source code
%cd planet/src
%ls
!qmake Tool.pro
!make

# Install Caffe - Currently not working in Python

Following [this tutorial](https://colab.research.google.com/github/Huxwell/caffe-colab/blob/main/caffe_details.ipynb). Caution: this takes 5 minutes.

We now install Caffe and Yices using apt.

In [ ]:
# Caffe currently doesnt work in Python, but you can train your model on cli.

%%capture

# install Caffe and Yices
# change root path of #CAFFE and #YICES
!sudo apt install caffe-cuda
!sudo add-apt-repository ppa:sri-csl/formal-methods -qq
!sudo apt-get update
!sudo apt-get install yices2

In [ ]:
%cd /content/
!git clone https://github.com/BVLC/caffe.git

We also install the needed libraries.

In [ ]:
%%capture 
!sudo apt-get install libgflags2.2 
!sudo apt-get install libgflags-dev
!sudo apt-get install libgoogle-glog-dev
!sudo apt-get install libhdf5-100
!sudo apt-get install libhdf5-serial-dev
!sudo apt-get install libhdf5-dev
!sudo apt-get install libhdf5-cpp-100
!sudo apt-get install libprotobuf-dev protobuf-compiler

In [ ]:
!find /usr -iname "*hdf5.so"
# got: /usr/lib/x86_64-linux-gnu/hdf5/serial
!find /usr -iname "*hdf5_hl.so"

To use the shared libraries for hdf5, we create symbolic links.

In [ ]:
!ln -s /usr/lib/x86_64-linux-gnu/libhdf5_serial.so /usr/lib/x86_64-linux-gnu/libhdf5.so
!ln -s /usr/lib/x86_64-linux-gnu/libhdf5_serial_hl.so /usr/lib/x86_64-linux-gnu/libhdf5_hl.so

We set the path for our HDF5 libs

In [ ]:
#!find /usr -iname "*hdf5.h*" # got:
# /usr/include/hdf5/serial/hdf5.h 
# /usr/include/opencv2/flann/hdf5.h
# Let's try the first one.
%env CPATH="/usr/include/hdf5/serial/"
#fatal error: hdf5.h: No such file or directory

In [ ]:
%%capture
!sudo apt-get install libleveldb-dev
!sudo apt-get install libgflags-dev libgoogle-glog-dev liblmdb-dev
!sudo apt-get install libsnappy-dev

Build caffe from source files.

In [ ]:
!echo $CPATH

We now change into the Coffe directory and build the shared Coffe libraries as well as the CPP object files

In [ ]:
%cd caffe

!ls
!make clean
!cp Makefile.config.example Makefile.config

In [ ]:
!sed -i 's/-gencode arch=compute_20/#-gencode arch=compute_20/' Makefile.config #old cuda versions won't compile 
!sed -i 's/\/usr\/local\/include/\/usr\/local\/include \/usr\/include\/hdf5\/serial\//'  Makefile.config #one of the 4 things needed to fix hdf5 issues
!sed -i 's/# OPENCV_VERSION := 3/OPENCV_VERSION := 3/' Makefile.config #We actually use opencv 4.1.2, but it's similar enough to opencv 3.
!sed -i 's/code=compute_61/code=compute_61 -gencode=arch=compute_70,code=sm_70 -gencode=arch=compute_75,code=sm_75 -gencode=arch=compute_75,code=compute_75/' Makefile.config #support for new GPUs

In [ ]:
!make all -j 4 # -j would use all availiable cores, but RAM related errors occur

We should now see the newly created shared libraries and Caffe object files.

In [ ]:
!find / -iname "*caffe*"

# Train Caffe model on MNIST

To train our model, we have to download the mnist dataset.

In [ ]:
# downloads mnist dataset

%cd /content/caffe/

!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz
!cp -rv MNIST/raw/* data/mnist/

In [ ]:
# creates mnist_test_lmdb and mnist_train_lmdb

!/content/caffe/examples/mnist/create_mnist.sh

Now we copy the output to the corresponding folder.

In [ ]:
# copy lmdbs to planet folder

%cp -a /content/caffe/examples/mnist/mnist_test_lmdb /content/planet/casestudies/MNIST/
%cp -a /content/caffe/examples/mnist/mnist_train_lmdb /content/planet/casestudies/MNIST/

### Define our neural network

Now we define the net and its structure with the individual layers, based on Planet's [example](https://github.com/progirep/planet/blob/master/casestudies/MNIST/lenet_train_test.prototxt).

Our net is a set of layers connected in a computation graph – a directed acyclic graph (DAG) to be exact. 
In Caffe, the net is defined as a set of layers and their connections in a plaintext modeling language. 

The net begins with a data layer that loads the MNIST data. After that, a reshape layer is used to change the dimensions of the input to match those of MNIST. Next up is a convolution layer and a corresponding pooling layer. A ReLU layer with 8 neurons followed by a softmax loss layer with 10 neuron gives us the output. 

The inner product layers in between are used to fully connect the layers. The accuracy is computed by the accuracy layer, this layer doesn't have a backward step.

In [ ]:
%%writefile /content/caffe/examples/mnist/lenet_train_test.prototxt
name: "LeNet"
layer {
  name: "mnist"
  type: "Data"
  top: "data"
  top: "label"
  include {
    phase: TRAIN
  }
  transform_param {
    scale: 0.00390625
  }
  data_param {
    source: "/content/caffe/examples/mnist/mnist_train_lmdb"
    batch_size: 64
    backend: LMDB
  }
}
layer {
  name: "mnist"
  type: "Data"
  top: "data"
  top: "label"
  include {
    phase: TEST
  }
  transform_param {
    scale: 0.00390625
  }
  data_param {
    source: "/content/caffe/examples/mnist/mnist_test_lmdb"
    batch_size: 100
    backend: LMDB
  }
}

layer {
    name: "reshapeA"
    type: "Reshape"
    bottom: "data"
    top: "reshapeA"
    reshape_param {
      shape {
        dim: -1  # copy the dimension from below
        dim: 1  # copy the dimension from below
        dim: 28  # copy the dimension from below
        dim: 28 # infer it from the other dimensions
      }
    }
}

layer {
  name: "conv1"
  type: "Convolution"
  bottom: "reshapeA"
  top: "conv1"
  param {
    lr_mult: 1
  }
  param {
    lr_mult: 2
  }
  convolution_param {
    num_output: 3
    kernel_size: 4
    stride: 2
    weight_filler {
      type: "xavier"
    }
    bias_filler {
      type: "constant"
    }
  }
}
layer {
  name: "pool1"
  type: "Pooling"
  bottom: "conv1"
  top: "pool1"
  pooling_param {
    pool: MAX
    kernel_size: 4
    stride: 3
  }
}

layer {
    name: "reshapeB"
    type: "Reshape"
    bottom: "pool1"
    top: "reshapeB"
    reshape_param {
      shape {
        dim: -1  # copy the dimension from below
        dim: 48 # infer it from the other dimensions
      }
    }
}

layer {
  name: "ip1"
  type: "InnerProduct"
  bottom: "reshapeB"
  top: "ip1"
  param {
    lr_mult: 1
  }
  param {
    lr_mult: 2
  }
  inner_product_param {
    num_output: 8
    weight_filler {
      type: "xavier"
    }
    bias_filler {
      type: "constant"
    }
  }
}
layer {
  name: "relu1"
  type: "ReLU"
  bottom: "ip1"
  top: "relu1"
}
layer {
  name: "ip2"
  type: "InnerProduct"
  bottom: "relu1"
  top: "ip2"
  param {
    lr_mult: 1
  }
  param {
    lr_mult: 2
  }
  inner_product_param {
    num_output: 10
    weight_filler {
      type: "xavier"
    }
    bias_filler {
      type: "constant"
    }
  }
}
layer {
  name: "accuracy"
  type: "Accuracy"
  bottom: "ip2"
  bottom: "label"
  top: "accuracy"
}
layer {
  name: "loss"
  type: "SoftmaxWithLoss"
  bottom: "ip2"
  bottom: "label"
  top: "loss"
}

### Define our training 

We will train our net on 10,000 images for 20,000 iterations with a variable, decaying learning rate.

In [ ]:
%%writefile /content/caffe/examples/mnist/lenet_solver.prototxt
# The train/test net protocol buffer definition
net: "examples/mnist/lenet_train_test.prototxt"
# test_iter specifies how many forward passes the test should carry out.
# In the case of MNIST, we have test batch size 100 and 100 test iterations,
# covering the full 10,000 testing images.
test_iter: 100
# Carry out testing every 500 training iterations.
test_interval: 1000
# The base learning rate, momentum and the weight decay of the network.
base_lr: 0.01
momentum: 0.9
weight_decay: 0.0005
# The learning rate policy
lr_policy: "inv"
gamma: 0.0001
power: 0.75
# Display every 100 iterations
display: 1000
# The maximum number of iterations
max_iter: 20000
# solver mode: CPU or GPU
solver_mode: CPU

### Train our model using Caffe

In [ ]:
# train the model

!/content/caffe/build/tools/caffe train --solver=/content/caffe/examples/mnist/lenet_solver.prototxt $@

The training results in a .caffemodel file (under /content/caffe/examples/mnist/lenet_solver_iter_20000.caffemodel), which now describes our trained model. To verify it with PLANET, we have to convert it to the right input format:

*.caffemodel -> .json -> .rlv*

# Convert Caffe model to Planet input file

Here we run into a problem; the Python scripts for converting the model are importing the caffe module, but in Colab this does not work. Caffe does not find the previously generated shared object file _caffe.so, which is wrapped in Python code using python 2.7. Therefore even changing the directory leads to a compatibility issue with the Python 2.7 compiled .so file and our Code in Python 3.

Therefore we have to download the corresponding file and run this offline. The corresponding output file can then be uploaded again and fed into the next steps. The Python functions can be found [here](https://github.com/DDiekmann/Applied-Verification-Lab-Neural-Networks/blob/main/lib/planet_helper_functions.py).

## Caffe to JSON converter in Python3

Run the function `caffeModelToJson()`.


## JSON to RLV converter in Python3

The next step is to convert the output in JSON format into the RLV format which can be read by planet. For this we simply use the script from the original author and rewrite it to work with python 3.

Run the function `jsonToRlv()`.

# Verify Robustness with Planet

Because the conversion doesn't work right away in Colab, we will download the files from GitHub. These files have been generated locally.

In [ ]:
!wget https://raw.githubusercontent.com/DDiekmann/Applied-Verification-Lab-Neural-Networks/main/lib/output.rlv -O /content/output.rlv
!wget https://raw.githubusercontent.com/DDiekmann/Applied-Verification-Lab-Neural-Networks/main/lib/caffemodel_mnist.json -O /content/caffemodel_output.json

In the converted rlv file we now add the input constraints for our verification

In [ ]:
# Add contraints on Input Variables for Planet
%cd /content/

with open("output.rlv", "ab") as f:
  for i in range(28*28):
    linebreak = bytes("\n", "utf-8")
    assert_lowerbound = bytes("Assert <= 0.0 1.0 inX" + str(i), "utf-8")
    assert_upperbound = bytes("Assert >= 1.0 1.0 inX" + str(i), "utf-8")

    f.write(linebreak)
    f.write(assert_lowerbound)
    f.write(linebreak)
    f.write(assert_upperbound)

In [ ]:
!pip install python-mnist

In [ ]:
from mnist import MNIST
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

data = MNIST("/content/caffe/data/mnist/")

imgs, labels = data.load_training()

img = np.asarray(imgs[0]).reshape(28, 28)
pixels = np.asarray(imgs[0])

plt.title("Label: {}".format(labels[0]))
plt.imshow(img)

In [ ]:
# targetDigit, maxDifferencePerPixel, maxUnsmoothnessInNoise 
# Obtain a digit image that is close to a given one, that resolves to the given target digit, 
# where every pixel is at most maxDifferencePerPixel away from the initial image 
# and the maximal noise difference between two adjacent pixels is maxUnsmoothnessInNoise. 
# The last two parameters should be >=0 and <=1 (such as, e.g., 0.05 for 5% deviation)"
maxDifferencePerPixel = 0.1
targetDigit = 5
maxUnsmoothnessInNoise = 0.05

def addBounds(i, l_bound, u_bound):
  with open("/content/output.rlv", "ab") as f:
    linebreak = bytes("\n", "utf-8")
    assert_lowerbound = bytes("Assert <= {} 1.0 inX{}".format(l_bound, i), "utf-8")
    assert_upperbound = bytes("Assert >= {} 1.0 inX{}".format(u_bound, i), "utf-8")

    f.write(linebreak)
    f.write(assert_lowerbound)
    f.write(linebreak)
    f.write(assert_upperbound)

# constraints for input neurons  
for i in range(28*28):
  # exclude outer pixels, because there are no neighbours
  x = i % 28
  y = int(i / 28)

  if x<3 or x>24 or y<3 or y>24:
    border = 0.0
  else:
    border = maxDifferencePerPixel

  lower_bound = max(0.0, pixels[i]/256.0 - border)
  upper_bound = min(1.0, pixels[i]/256.0 + border) 
  addBounds(i, lower_bound, upper_bound)

# constraints for output neurons
for i in range(10):
  if i == targetDigit:
    continue
  with open("/content/output.rlv", "ab") as f:
    assertion = "\nAssert >= -0.000001 1.0 outX{} -1.0 outX{}".format(i, targetDigit)
    f.write(bytes(assertion, "utf-8"))

# constraints for smoothness
for x in range(28):
  for y in range(28):
    if y < 27:
      pixelDiff = (pixels[y*28+x] - pixels[(y+1)*28+x]) / 256.0
      ass1 = "\nAssert <= {} 1.0 inX{} -1.0 inX{}".format((pixelDiff-maxUnsmoothnessInNoise), (y*28+x), (y+1)*28+x)
      ass2 = "\nAssert >= {} 1.0 inX{} -1.0 inX{}".format((pixelDiff+maxUnsmoothnessInNoise), (y*28+x), (y+1)*28+x)
      with open("/content/output.rlv", "ab") as f:
        f.write(bytes(ass1, "utf-8"))
        f.write(bytes(ass2, "utf-8"))
    if x < 27: 
      pixelDiff = (pixels[y*28+x] - pixels[y*28+x+1]) / 256.0
      ass1 = "\nAssert <= {} 1.0 inX{} -1.0 inX{}".format((pixelDiff-maxUnsmoothnessInNoise), (y*28+x), (y*28+x+1))
      ass2 = "\nAssert >= {} 1.0 inX{} -1.0 inX{}".format((pixelDiff+maxUnsmoothnessInNoise), (y*28+x), (y*28+x+1))
      with open("/content/output.rlv", "ab") as f:
        f.write(bytes(ass1, "utf-8"))
        f.write(bytes(ass2, "utf-8"))

print("FINISHED ADDING CONSTRAINTS!")

Run planet and save the output in a text file, so we can parse it for values later. This has to be run twice, because Colab returns an error the first time.

In [ ]:
%%capture cap --no-stderr
!/content/planet/src/planet /content/output.rlv

with open("/content/planet_output.txt", "w") as f:
  f.write(cap.stdout)

In [ ]:
sat = False
valLineFound = False
values = {}

# parse the planet output
with open("/content/planet_output.txt", "r") as f:
  for line in f.readlines():
    line = line.strip()

    if line == "SAT":
      sat = True
    elif line == "Valuation:":
      valLineFound = True
    elif line.startswith("- ") and valLineFound:
      parts = line.split(" ")
      
      assert parts[0] == "-"
      assert parts[3] == "/"

      # DEBUG prints
      # print(parts[1][:len(parts[1])-1])
      # print(parts[2])
      # break
      # builds a dictionary with the calculated values
      # e.g. values[inX0] = 0.0
      values[parts[1][:len(parts[1])-1]] = float(parts[2])

# create adverserial example
if sat:
  outImg = Image.new("L", (28, 28))
  for y in range(28):
    for x in range(28):
      outImg.putpixel((x, y), int(256*values["inX{}".format(y*28 + x)]))

  #plt.title("Label: {}".format(labels[0]))
  plt.imshow(outImg)

This is our adversarial example.

If you want to try different Asserts, the next cell removes all Asserts from the output.rlv file.

In [ ]:
# removes assert conditions and empty lines
!sed -i "/Assert/d" /content/output.rlv
!sed -i "/^$/d" /content/output.rlv 